# Grover's Algorithm, First Demo
Daniel Evans

In [69]:
# imports
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit import QuantumCircuit,transpile,assemble
from qiskit.circuit.library import MCXGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator

from qiskit_aer import Aer

import numpy as np

In [70]:
U = np.array([
	[1,0,0,0,0,0,0,0],
	[0,1,0,0,0,0,0,0],
	[0,0,1,0,0,0,0,0],
	[0,0,0,-1,0,0,0,0],
	[0,0,0,0,1,0,0,0],
	[0,0,0,0,0,1,0,0],
	[0,0,0,0,0,0,1,0],
	[0,0,0,0,0,0,0,1],
])

In [71]:
circuit = QuantumCircuit(3)
#circuit.draw()

circuit.h([0,1,2])
#circuit.draw()

# - ITERATE ORACLE AND DIFFUSION -
for i in range(2):
	circuit.unitary(U,[0,1,2],"?")
	#circuit.cz(1,2)
	circuit.draw()
	#circuit.decompose().draw()

	circuit.h([0,1,2])
	circuit.x([0,1,2])

	# multi-controlled z gate or smthn ?????
	circuit.h(2)
	circuit.append(MCXGate(2),[0,1,2])  # multi-controlled-toffoli
	circuit.h(2)

	circuit.x([0,1,2])
	circuit.h([0,1,2])


circuit.measure_all()
circuit.draw()

┌───┐┌────┐┌───┐┌───┐          ┌───┐┌───┐     ┌────┐┌───┐┌───┐     »
   q_0: ┤ H ├┤0   ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤0   ├┤ H ├┤ X ├─────»
        ├───┤│    │├───┤├───┤       │  ├───┤├───┤     │    │├───┤├───┤     »
   q_1: ┤ H ├┤1 ? ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤1 ? ├┤ H ├┤ X ├─────»
        ├───┤│    │├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐│    │├───┤├───┤┌───┐»
   q_2: ┤ H ├┤2   ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤2   ├┤ H ├┤ X ├┤ H ├»
        └───┘└────┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└────┘└───┘└───┘└───┘»
meas: 3/═══════════════════════════════════════════════════════════════════»
                                                                           »
«             ┌───┐┌───┐      ░ ┌─┐      
«   q_0: ──■──┤ X ├┤ H ├──────░─┤M├──────
«          │  ├───┤├───┤      ░ └╥┘┌─┐   
«   q_1: ──■──┤ X ├┤ H ├──────░──╫─┤M├───
«        ┌─┴─┐├───┤├───┤┌───┐ ░  ║ └╥┘┌─┐
«   q_2: ┤ X ├┤ H ├┤ X ├┤ H ├─░──╫──╫─┤M├
«        └───┘└───┘└───┘└───┘ ░  ║  ║ └╥┘
«meas: 3/════════════════════════╩══╩══╩═
«                                0  1  2

In [72]:
# Simulating
sim = Aer.get_backend('qasm_simulator')
transpiled_circuit = transpile(circuit, sim)
results = sim.run(transpiled_circuit).result()
counts = results.get_counts()
print(counts)

{'011': 969, '000': 8, '101': 10, '001': 7, '110': 8, '111': 13, '100': 5, '010': 4}


In [73]:
token_ = open("../../token.txt",'r',encoding='utf-16').read()
service = QiskitRuntimeService(channel="ibm_quantum", token=token_)
 
backend = service.least_busy(simulator=False, operational=True)
 
print("Optimizing...")
# Convert to an ISA circuit and layout-mapped observables.
transpiledcircuit = transpile(circuit, backend)
 
# One pub, with one circuit to run against five different observables.
job = backend.run(transpiledcircuit,shots=5000)
 
# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")

Optimizing...


C:\Users\danev\AppData\Local\Temp\ipykernel_34116\290281285.py:11: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(transpiledcircuit,shots=5000)


>>> Job ID: cw2krkarxqkg008e66zg


In [67]:
job = service.job("cw2a85r54nq0008x8vrg")
job.cancel()